## Run evaluation on different action policies, e.g. VLA

In [ ]:
from VLABench.evaluation.evaluator import Evaluator
from VLABench.evaluation.model.policy.openvla import OpenVLA
from VLABench.evaluation.model.policy.base import RandomPolicy
from VLABench.tasks import *
from VLABench.robots import *

demo_tasks = ["select_fruit"]
unseen = True
save_dir = "/home/shiduo/project/VLABench/logs"

model_ckpt = "/remote-home1/pjliu/openvla-7b"
lora_ckpt = "/remote-home1/pjliu/openvla/weights/select_fruit+CSv1+lora/"

In [ ]:
import os
os.environ["MUJOCO_GL"] = "egl"

### Init evaluator

In [ ]:
evaluator = Evaluator(
    tasks=demo_tasks,
    n_episodes=2,
    max_substeps=10,   
    save_dir=save_dir,
    visulization=True
)

### Load basic random policy

In [ ]:
random_policy = RandomPolicy(model=None)
result = evaluator.evaluate(random_policy)

### Load policies, take OpenVLA as example

In [ ]:
policy = OpenVLA(
    model_ckpt=model_ckpt,
    lora_ckpt=lora_ckpt,
    norm_config_file=os.path.join(os.getenv("VLABENCH_ROOT"), "configs/model/openvla_config.json")
)

result = evaluator.evaluate(policy)

## Run evaluation on different VLMs

In [ ]:
from VLABench.evaluation.model.vlm import *
from VLABench.evaluation.evaluator import VLMEvaluator

vlm_name = "GPT_4v" # valid names: ["GPT_4v", "Qwen2_VL", "InternVL2", "MiniCPM_V2_6", "GLM4v", "Llava_NeXT"]
fewshot_num = 0
task_list = ["mesh_and_texture/select_fruit"]

def initialize_model(model_name, *args, **kwargs):
    cls = globals().get(model_name)
    if cls is None:
        raise ValueError(f"Model '{model_name}' not found in the current namespace.")
    
    return cls(*args, **kwargs)


In [ ]:
vlm = initialize_model(vlm_name)
evaluator = VLMEvaluator(
    tasks=task_list,
    n_episodes=2,
    data_path=os.path.join(os.getenv("VLABENCH_ROOT"), "../dataset", "vlm"),
    save_path=os.path.join(os.getenv("VLABENCH_ROOT"), "../logs/vlm"),
)

evaluator.evaluate(vlm, few_shot_num=fewshot_num)
result=evaluator.get_final_score_dict(vlm_name)
